In [70]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import argparse
import os

# stuff just for IPython
from IPython.display import display
from IPython.display import Image
from matplotlib import pyplot as plt
from skimage import filters
%matplotlib inline

### Stitcher class

This is a class provided by OpenCV that achieves basic left-to-right stitching.

In [71]:
class Stitcher:
	def __init__(self):
		# determine if we are using OpenCV v3.X
		self.isv3 = imutils.is_cv3()

	def stitch(self, images, ratio=0.75, reprojThresh=4.0,
		showMatches=False):
		# unpack the images, then detect keypoints and extract
		# local invariant descriptors from them
		(imageB, imageA) = images
		(kpsA, featuresA) = self.detectAndDescribe(imageA)
		(kpsB, featuresB) = self.detectAndDescribe(imageB)

		# match features between the two images
		M = self.matchKeypoints(kpsA, kpsB,
			featuresA, featuresB, ratio, reprojThresh)

		# if the match is None, then there aren't enough matched
		# keypoints to create a panorama
		if M is None:
			return None

		# otherwise, apply a perspective warp to stitch the images
		# together
		(matches, H, status) = M
		result = cv2.warpPerspective(imageA, H,
			(imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
		result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

		# check to see if the keypoint matches should be visualized
		if showMatches:
			vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,
				status)

			# return a tuple of the stitched image and the
			# visualization
			return (result, vis)

		# return the stitched image
		return result

	def detectAndDescribe(self, image):
		# convert the image to grayscale
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

		# check to see if we are using OpenCV 3.X
		if self.isv3:
			# detect and extract features from the image
			descriptor = cv2.xfeatures2d.SIFT_create()
			(kps, features) = descriptor.detectAndCompute(image, None)

		# otherwise, we are using OpenCV 2.4.X
		else:
			# detect keypoints in the image
			detector = cv2.FeatureDetector_create("SIFT")
			kps = detector.detect(gray)

			# extract features from the image
			extractor = cv2.DescriptorExtractor_create("SIFT")
			(kps, features) = extractor.compute(gray, kps)

		# convert the keypoints from KeyPoint objects to NumPy
		# arrays
		kps = np.float32([kp.pt for kp in kps])

		# return a tuple of keypoints and features
		return (kps, features)

	def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
		ratio, reprojThresh):
		# compute the raw matches and initialize the list of actual
		# matches
		matcher = cv2.DescriptorMatcher_create("BruteForce")
		rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		matches = []

		# loop over the raw matches
		for m in rawMatches:
			# ensure the distance is within a certain ratio of each
			# other (i.e. Lowe's ratio test)
			if len(m) == 2 and m[0].distance < m[1].distance * ratio:
				matches.append((m[0].trainIdx, m[0].queryIdx))

		# computing a homography requires at least 4 matches
		if len(matches) > 4:
			# construct the two sets of points
			ptsA = np.float32([kpsA[i] for (_, i) in matches])
			ptsB = np.float32([kpsB[i] for (i, _) in matches])

			# compute the homography between the two sets of points
			(H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC,
				reprojThresh)

			# return the matches along with the homograpy matrix
			# and status of each matched point
			return (matches, H, status)

		# otherwise, no homograpy could be computed
		return None

	def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
		# initialize the output visualization image
		(hA, wA) = imageA.shape[:2]
		(hB, wB) = imageB.shape[:2]
		vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
		vis[0:hA, 0:wA] = imageA
		vis[0:hB, wA:] = imageB

		# loop over the matches
		for ((trainIdx, queryIdx), s) in zip(matches, status):
			# only process the match if the keypoint was successfully
			# matched
			if s == 1:
				# draw the match
				ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
				ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
				cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

		# return the visualization
		return vis

In [92]:
def choose_homography(img1,img2,H):
    result = cv2.warpPerspective(img2, H,
                                (img1.shape[0] + 2*img2.shape[0], 2*img2.shape[1]+img1.shape[1]))
    img1_bin = cv2.threshold(img1,0,1,cv2.THRESH_BINARY_INV)[1]
    result[0:img1.shape[0], 0:img1.shape[1]] = result[0:img1.shape[0], 0:img1.shape[1]]*img1_bin+img1

    for r in range(0,result.shape[0]):
        if not sum(result[r,:,0]):
            r_bound =r;
            break
    for c in range(0,result.shape[1]):
        if not sum(result[:,c,0]):
            c_bound =c;
            break
    if c == result.shape[1]-1:
        c_bound = c
    if r == result.shape[0]-1:
        r_bound = r
            
    return result[0:r_bound,0:c_bound]

In [93]:
get_fnames = lambda f_name: list(map(lambda x:f_name+x,os.listdir(f_name)))
get_img_set = lambda f_names:[cv2.imread(name) for name in get_fnames(f_names)]
scale_img_set = lambda scale:[cv2.res]

In [100]:
img_set = get_img_set('test2/')

In [97]:
# test sequence 2: run stitcher on two images #
stitcher = Stitcher()
img = img_set[0]
for j in range(1,6):
    kp1,f1 = stitcher.detectAndDescribe(img)
    kp2,f2 = stitcher.detectAndDescribe(img_set[j])
    M = stitcher.matchKeypoints(kp1, kp2, f1, f2,0.75,4)
    (matches, H, status) = M
    img = choose_homography(img,img_set[j],H)

In [99]:
cv2.imwrite('end.jpg',img)

True